## 年齢を類推するコトを考える。  
## 精度は必要ないので、10歳刻みに分類する  
## 料金も精度は必要ないので、20ドル刻みくらいで分類する。  
## 年齢とsurvived以外を説明変数、年齢を目的変数にするのはどうか。

In [21]:
import numpy as np
import pandas as pd

train = pd.read_csv('data/train.tsv',delimiter='\t')

train2 = train[np.isnan(train['age'])==False]
test = train[np.isnan(train['age'])==True]

In [5]:
#age_category=[0,10,20,30,40,50,60,70,80,90]
#train2['age2']=pd.cut(train2['age'],age_category)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [6]:
train2.head()

,id,survived,pclass,sex,age,sibsp,parch,fare,embarked,age2
0,3,1,1,female,35.0,1,0,53.1000,S,"(30, 40]"
1,4,0,3,male,35.0,0,0,8.0500,S,"(30, 40]"
2,7,0,3,male,2.0,3,1,21.0750,S,"(0, 10]"
3,9,1,2,female,14.0,1,0,30.0708,C,"(10, 20]"
4,11,1,1,female,58.0,0,0,26.5500,S,"(50, 60]"


#### ーーーなんと、age2が思いもよらない値。単純な値になって欲しかった。  
#### やはり、自前でカテゴリ化するようなlamda式を書くしかないか。

In [37]:
t = train2['age'].apply(lambda x: int(x /10) )

In [38]:
t2=pd.concat([train2,t],axis=1)

In [47]:
t2.columns=['id','survived','pclass','sex','age','sibsp','parch','fare','embarked','age2']

In [48]:
t2.head()

,id,survived,pclass,sex,age,sibsp,parch,fare,embarked,age2
0,3,1,1,female,35.0,1,0,53.1000,S,3
1,4,0,3,male,35.0,0,0,8.0500,S,3
2,7,0,3,male,2.0,3,1,21.0750,S,0
3,9,1,2,female,14.0,1,0,30.0708,C,1
4,11,1,1,female,58.0,0,0,26.5500,S,5


In [53]:
t3=t2['fare'].apply(lambda x: int(x/100*10))

In [54]:
t4=pd.concat([t2,t3],axis=1)

In [55]:
t4.head()

,id,survived,pclass,sex,age,sibsp,parch,fare,embarked,age2,fare
0,3,1,1,female,35.0,1,0,53.1000,S,3,5
1,4,0,3,male,35.0,0,0,8.0500,S,3,0
2,7,0,3,male,2.0,3,1,21.0750,S,0,2
3,9,1,2,female,14.0,1,0,30.0708,C,1,3
4,11,1,1,female,58.0,0,0,26.5500,S,5,2


In [58]:
t5=pd.concat([t4,pd.get_dummies(t4['sex']),pd.get_dummies(t4['embarked'])],axis=1)

In [59]:
t5.head()

,id,survived,pclass,sex,age,sibsp,parch,fare,embarked,age2,fare,female,male,C,Q,S
0,3,1,1,female,35.0,1,0,53.1000,S,3,5,1,0,0,0,1
1,4,0,3,male,35.0,0,0,8.0500,S,3,0,0,1,0,0,1
2,7,0,3,male,2.0,3,1,21.0750,S,0,2,0,1,0,0,1
3,9,1,2,female,14.0,1,0,30.0708,C,1,3,1,0,1,0,0
4,11,1,1,female,58.0,0,0,26.5500,S,5,2,1,0,0,0,1


In [60]:
t5.columns=['id','survived','pclass','sex','age','sibsp','parch','fare','embarked','age2','fare2','female','male','C','Q','S']

In [64]:
y=t5['age2']
x=t5[['pclass','sibsp','parch','fare2','female','male','C','Q','S']]

In [67]:
from sklearn.model_selection import train_test_split
x_train, x_train_valid, y_train, y_train_valid = train_test_split(x,y, test_size=0.3)

In [105]:
from sklearn.tree import DecisionTreeClassifier
model=DecisionTreeClassifier(random_state=0,max_depth=7)
model.fit(x_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=7,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [106]:
y_predicted=model.predict(x_train)
y_predicted_valid=model.predict(x_train_valid)

In [107]:
(y_predicted==y_train).sum()/len(y_train)

0.61904761904761907

In [108]:
(y_predicted_valid==y_train_valid).sum()/len(y_train_valid)

0.34259259259259262

### ここまでで年齢類推用のmodelができた。これを使って年齢を埋める